# This project implemented by Elham Mahdipour
## She is a Ph.D. Candidate of computer engineering at Yazd University, Yazd, Iran.
### Please feel free and contact to me: elham.mahdipour@gmail.com/ elham.mahdipour@stu.yazd.ac.ir

# Phase 1 
## Create Dataset and Preprocessing

In [1]:
import networkx as nx
G1=nx.read_leda('biogrid samples\RNorvegicus.gw')
G1

In [2]:
G2=nx.read_leda('biogrid samples\MMusculus.gw')
G2

In [3]:
### Check and Swap if G1 > G2 ###
print(len(G1))
print(len(G2))
print('*******')
if len(G1)>len(G2):
    temp=G1
    G1=G2
    G2=temp
print(len(G1))
print(len(G2))

1657
4370
*******
1657
4370


In [4]:
G_target_na=nx.read_weighted_edgelist('biogrid samples\RN-MM.tsv')
print(len(G_target_na))
G_target_na  

34042


In [5]:
ed1=G1.edges()
ed2=G2.edges()

nd1=G1.nodes()
nd2=G2.nodes()

el1=list(ed1)
el2=list(ed2)

nd1=list(nd1)
nd2=list(nd2)

degG1 = [val for (node, val) in G1.degree()]
degG2 = [val for (node, val) in G2.degree()]

# Feature Extraction

# compute score for create similarity matrix

In [6]:
def deg_Diff(G1,G2):
    Degree_Difference=np.zeros((len(G1),len(G2)))
    for i in range(len(G1)):
        for j in range(len(G2)):
            Degree_Difference[i][j]=abs(degG1[i]-degG2[j])/max(degG1[i],degG2[j])
    return Degree_Difference

In [7]:
def score_pageRank(X):
    a=nx.pagerank(X)
    return a

In [8]:
def coefficient_pagerank(x,y):  #x is G1, y is G2
   # print(len(x))
    p1=score_pageRank(x)
    b=p1.values()
    pr1=list(b)
    p2=score_pageRank(y)
    c=p2.values()
    pr2=list(c)
    pr=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            #print(pr1[i],pr2[j])
            pr[i][j]=abs(pr1[i]-pr2[j])/max(pr1[i],pr2[j])   #minimum pr is maximum similarity of topology 
    return pr

In [9]:
def coefficient_edges(index_node, G, GraphNumber):
    if GraphNumber==1:
        sum_edge=0        
        for i in G.neighbors(nd1[index_node]):                       
            sum_edge=sum_edge+degG1[nd1.index(i)]
        #print(sum_edge)
        temp=(degG1[index_node]-1) if degG1[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG1[index_node]*temp)
    else:
        sum_edge=0        
        for i in G.neighbors(nd2[index_node]):                       
            sum_edge=sum_edge+degG2[nd2.index(i)]
        #print(sum_edge)
        temp=(degG2[index_node]-1) if degG2[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG2[index_node]*temp)
    return coeff_node

In [10]:
def compute_Ea(G1,G2):
    Ea_G1=np.zeros(len(G1))
    Ea_G2=np.zeros(len(G2))
    for i in range(len(G1)):
        Ea_G1[i]=coefficient_edges(i, G1, 1)
    for j in range(len(G2)):
        Ea_G2[j]=coefficient_edges(j,G2,2)
    ea=[Ea_G1, Ea_G2]
    return(ea)

In [11]:
# compute relative clustering coefficient difference between node a (in G1) and node b (in G2)
def CD(G1, G2):
    cd=np.zeros((len(G1),len(G2)))
    EA=compute_Ea(G1,G2)
    #print(EA[0])      #Ea for G1
    #print("===================")
    #print(EA[1])      #Ea for G2
    for i in range(len(G1)):
        for j in range(len(G2)):
            cd[i,j]=abs(EA[0][i]-EA[1][j])/max(EA[0][i],EA[1][j])
    return cd

In [12]:
elt=G_target_na.edges()
elt=list(elt)
print(len(elt))
elt

17021


[('360863', '226551'),
 ('24691', '22072'),
 ('361791', '106582'),
 ('117287', '57320'),
 ('252881', '211446'),
 ('100910650', '381043'),
 ('64831', '64602'),
 ('29254', '23945'),
 ('289076', '64214'),
 ('300993', '109685'),
 ('314690', '319277'),
 ('102548230', '56525'),
 ('291952', '72361'),
 ('50681', '11430'),
 ('313993', '195733'),
 ('690037', '64931'),
 ('688903', '75533'),
 ('100364165', '18087'),
 ('301121', '19725'),
 ('305589', '14025'),
 ('680244', '109218'),
 ('288704', '72357'),
 ('305317', '231279'),
 ('680177', '100504446'),
 ('24267', '12846'),
 ('294012', '66439'),
 ('361107', '56249'),
 ('360942', '54216'),
 ('315347', '74174'),
 ('678772', '213819'),
 ('84005', '217715'),
 ('361087', '105439'),
 ('498678', '72454'),
 ('360483', '73261'),
 ('361588', '67867'),
 ('308708', '67009'),
 ('361846', '30930'),
 ('50671', '14104'),
 ('363972', '269109'),
 ('291047', '107889'),
 ('308067', '105246'),
 ('25428', '13122'),
 ('303580', '71059'),
 ('102552939', '75577'),
 ('499020

In [13]:
def sequence_score(x,y):
    seq=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            q1=(nd1[i],nd2[j]) in elt
            if q1==False:
                c=0
            else:
               # print(q1)
                c=1
            seq[i][j]=c    
            
    return seq

In [14]:
def compute_score(G1,G2):
    coeff_pr=coefficient_pagerank(G1,G2)
    dd=deg_Diff(G1,G2)
    cd=CD(G1,G2)
    seq_sc=sequence_score(G1,G2)
    
    alpha=0.1
    betta=0.2
    gamma=0.2
    zetta=1-alpha-betta-gamma
    s=alpha*(1-coeff_pr)+betta*(1-dd)+gamma*(1-cd)+zetta*seq_sc
    return s,coeff_pr, dd, cd, seq_sc

In [15]:
import numpy as np
m, coeff_pr, dd, cd, seq=compute_score(G1,G2)
sim=m

# Phase 2: Dataset Generation

# create data for deep learning

# change problem to classification 
## [node of G1, node of G2, sequence similarity, Coefficient page rank, clustering coefficient difference,  similarity score, alignment=yes(0)/TN or no(1)/TP]

In [16]:
en_mat=[]

for i in range(len(nd1)):
    for j in range(len(nd2)):
        if G_target_na.has_edge(nd1[i],nd2[j]):
            align_class='Yes'
        else:
            align_class='No'
        
        sample=[int(nd1[i]),int(nd2[j]),seq[i][j], coeff_pr[i][j], dd[i][j],cd[i][j],sim[i][j],align_class] 
        en_mat.append(sample)
print(len(en_mat))

7241090


In [17]:
yc=[]
noc=[]
for i in range(len(en_mat)):
    if en_mat[i][7]=='Yes': 
        yc.append(en_mat[i])
    else:
        noc.append(en_mat[i])
print(len(yc), len(noc))

916 7240174


In [18]:
data=yc+noc 
len(data)
X=[]
y=[]
for i in range(len(data)):
    X.append(data[i][0:7])
    y.append(data[i][7])
    
len(X),len(y)

(7241090, 7241090)

In [19]:
X_train=X
y_train=y

In [20]:
x_tr,y_tr=X,y

# test model for real data

In [21]:
# Label encode Class (Species)
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# One Hot Encode
y_train = np_utils.to_categorical(encoded_Y)

Using TensorFlow backend.


In [22]:
import numpy as np
X_train=np.array(X_train)
y_train=np.array(y_train)

# Phase 3: LSTM-NA network

## Load tune LSTM-NA model

In [23]:
from keras.models import load_model
#del model_onelstm
model_onelstm = load_model('biogrid samples/deep_model_resample_lstm_SC-AT.h5')
model_onelstm.load_weights('biogrid samples/deep_model_resample_lstm_weights_SC-AT.h5', by_name=True)

In [24]:
# without resample
result_tr = model_onelstm.evaluate(X_train, y_train)
result_tr

7241090/7241090 [==============================] - 484s 67us/step


[6.914491689419227e-06, 1.0, 6.916883648955263e-06, 4.813336187958761e-11]

In [66]:
%%time
out_tr=model_onelstm.predict(X_train)
out_tr

Wall time: 5min 31s


In [77]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train.argmax(axis=1), out_tr.argmax(axis=1)))

[[7240174       0]
 [      0     916]]


In [69]:
from sklearn.metrics import f1_score, precision_score, recall_score
print(precision_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(recall_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(f1_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))

1.0
1.0
1.0


In [70]:
from sklearn import metrics

# Create alignment file 
f = open("biogrid samples/align-RN-MM.txt", "w")
for i in range(len(out_tr)):
    if out_tr[i][1]>out_tr[i][0]:
        z=[X_train[i][0],X_train[i][1]]
        f.write(str(z))
f.close()


print(metrics.roc_auc_score(y_train, out_tr))
print(metrics.matthews_corrcoef(y_train.argmax(axis=1),out_tr.argmax(axis=1)))

1.0
1.0


# test other classifier without resample

In [81]:
x_tra,y_tra=X_train,y_train

In [82]:
X_train,y_train=x_tr,y_tr

In [85]:
%%time
# Linear Discriminant Analysis

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(X_train, y_train)))

from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = lda.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Accuracy of LDA classifier on training set: 1.00
['Yes' 'No' 'Yes' ... 'No' 'No' 'No']
0.9998955958288048
[[7240174       0]
 [    756     160]]
0.9999477967609134
0.5873362445414847
0.6486727817761607
Wall time: 58.1 s


In [86]:
%%time
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = knn.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

Accuracy of KNN classifier on training set: 1.00
['No' 'No' 'No' ... 'No' 'No' 'No']
0.9998736378086724
[[7240174       0]
 [    915       1]]
0.9999368188956108
0.5005458515283843
0.5010589199926476
Wall time: 6min 38s


In [87]:
%%time
# SVM
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
print('Accuracy of SVM classifier on training set: {:.2f}'
     .format(svm.score(X_train, y_train)))

from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = svm.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

Accuracy of SVM classifier on training set: 1.00
['No' 'No' 'No' ... 'No' 'No' 'No']
0.9996473816924672
[[7240174       0]
 [    916       0]]
0.4999368126956132
0.5013456213783431
0.4910589187526434
Wall time: 32min 46s
